In [ ]:
import json
import boto3
import logging
import traceback
import time

In [ ]:
# === Configure CloudWatch logger ===
logger = logging.getLogger()
logger.setLevel(logging.INFO)

def lambda_handler(event, context):
    """
    Lambda: Investment decision synthesis for S&P 500 based on AI analysis

    Expected event:
    {
        "sp500_analysis": {...},  # output of your S&P500 model (with recommendations, sectors, etc.)
        "prompt_mode": "summary|detailed|executive",
        "language": "fr|en"
    }

    Returns:
    {
        "statusCode": int,
        "body": {
            "summary": str,
            "recommendations": str,
            "metadata": {...}
        }
    }
    """

    start_time = time.time()
    logger.info("=== Lambda execution started ===")
    logger.info(f"Incoming event: {json.dumps(event, ensure_ascii=False)}")

    try:
        # === Validate input ===
        if not isinstance(event, dict):
            raise ValueError("Event must be a JSON object")

        analysis_data = event.get("sp500_analysis")
        if not analysis_data:
            logger.warning("Missing field 'sp500_analysis' in event")
            return _error_response(400, "Missing field 'sp500_analysis' in event payload")

        prompt_mode = event.get("prompt_mode", "summary")
        language = event.get("language", "fr")

        logger.info(f"Prompt mode: {prompt_mode}, Language: {language}")

        # === Initialize Bedrock client ===
        try:
            bedrock = boto3.client("bedrock-runtime", region_name="us-east-1")
            logger.info("Bedrock client initialized successfully")
        except Exception as e:
            logger.error(f"Failed to initialize Bedrock client: {e}")
            return _error_response(500, f"Failed to initialize Bedrock client: {str(e)}")

        # === Compose prompt ===
        try:
            base_prompt = f"""
Tu es un analyste financier senior spécialisé en stratégie d'investissement.
Résume et interprète les résultats d'analyse du S&P500 ci-dessous.

Mode: {prompt_mode}
Langue: {language}

Analyse à interpréter :
{json.dumps(analysis_data, indent=2, ensure_ascii=False)}

Objectifs :
1. Résumer la situation globale du marché
2. Identifier les secteurs gagnants/perdants
3. Proposer une stratégie d’investissement claire et concise
4. Mettre en avant les risques principaux à surveiller
5. Terminer avec une conclusion professionnelle (ton analyste)
"""
            logger.debug(f"Generated prompt: {base_prompt[:500]}...")  # partial preview
        except Exception as e:
            logger.error(f"Prompt composition failed: {e}")
            return _error_response(400, f"Failed to build prompt: {str(e)}")

        # === Bedrock model call ===
        try:
            response = bedrock.invoke_model(
                modelId="anthropic.claude-3-sonnet-20240229-v1:0",
                body=json.dumps({
                    "anthropic_version": "bedrock-2023-05-31",
                    "max_tokens": 1000,
                    "messages": [
                        {"role": "user", "content": base_prompt}
                    ]
                })
            )
            logger.info("Model invoked successfully")
        except Exception as e:
            logger.error(f"Bedrock model invocation failed: {e}")
            return _error_response(502, f"Bedrock model invocation failed: {str(e)}")

        # === Parse response ===
        try:
            body = json.loads(response["body"].read())
            text_out = body.get("content", [{}])[0].get("text", "")
            if not text_out:
                raise ValueError("Empty response content from model")
            logger.info("Model output parsed successfully")
        except Exception as e:
            logger.error(f"Error parsing model output: {e}")
            return _error_response(500, f"Error parsing model output: {str(e)}")

        # === Success response ===
        execution_time = round(time.time() - start_time, 2)
        logger.info(f"Lambda execution completed successfully in {execution_time}s")

        return {
            "statusCode": 200,
            "body": json.dumps({
                "summary": "Synthèse stratégique générée avec succès",
                "recommendations": text_out.strip(),
                "metadata": {
                    "model_used": "anthropic.claude-3-sonnet-20240229-v1:0",
                    "prompt_mode": prompt_mode,
                    "language": language,
                    "execution_time_seconds": execution_time
                }
            }, ensure_ascii=False)
        }

    except ValueError as e:
        logger.warning(f"ValueError: {str(e)}")
        return _error_response(400, str(e))
    except Exception as e:
        logger.error(f"Unhandled error: {str(e)}")
        traceback_str = traceback.format_exc()
        logger.error(traceback_str)
        return _error_response(500, f"Unhandled error: {str(e)}", details=traceback_str)


# === Helper for structured error responses ===
def _error_response(code, message, details=None):
    logger.error(f"Returning error response [{code}]: {message}")
    return {
        "statusCode": code,
        "body": json.dumps({
            "error": {
                "code": code,
                "message": message,
                "details": details
            }
        }, ensure_ascii=False)
    }
